In [118]:
from sklearn import datasets
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split

In [119]:
iris = datasets.load_iris()
type(iris)

X = iris.data[50:,:2]
Y = iris.target[50:]

In [120]:
X_train, X_test, y_train, y_test = train_test_split(X, Y, train_size=0.8, random_state=42)

In [121]:
m, n = X_train.shape
print(f'Число объектов в обучающей выборке: {m}\nРазмерность объекта: {n}')

Число объектов в обучающей выборке: 80
Размерность объекта: 2


Логистическая фунция потерь:

In [122]:
def log_loss(y_true, y_pred):
    return -np.sum(y_true * np.log(y_pred) + (1 - y_true) * np.log(1 - y_pred), axis=0) / len(y_true)

Сигмоида:

In [123]:
def sigmoid(x):
    return 1 / (1 + np.exp(-x))

минимизируем функцию потерь с помощью градиентного спуска и RMSProp:

In [175]:
class LogisticRegression:
    
    def __init__(self):
        self.w = np.random.randn(n, 1) * 0.001
        self.b = np.random.randn() * 0.001
        self.report_every = 10
                
    def train_grad(self, X, y, learning_rate=0.001, epochs=100):
        self.losses_train = []
        self.losses_test = []
        
        for epoch in range(epochs):            
            Z = X.reshape(m, n).dot(self.w) + self.b
            A = sigmoid(Z)
            
            dw = np.sum(X.reshape(m, n) * (A.reshape(m, 1) - y.reshape(m, 1)), axis=0) / len(X)
            db = np.sum((A.reshape(m, 1) - y.reshape(m, 1)), axis=0) / len(X)
            
            # gradient step
            self.w = self.w - learning_rate * dw.reshape(n, 1)
            self.b = self.b - learning_rate * db
            
            # save loss for plotting
            if epoch % self.report_every == 0:
                self.losses_train.append(log_loss(y, self.predict(X)))
                self.losses_test.append(log_loss(y_test, self.predict(X_test)))
    
    def train_RMSProp(self, X, y, learning_rate=0.001, epochs=100, gamma=0.9, lr=0.25, eps=0.00001):
        
        self.losses_train = []
        self.losses_test = []
        
        for epoch in range(epochs):            
            dw = np.zeros((n, 1))
            dw = np.array([[1.], [1.]])
            db = 1
            EG = 0
            
            for i in range(len(X)):
                # forward propagation
                z = X[i].reshape(1, n).dot(self.w) + self.b
                a = sigmoid(z)[0][0]
                
                # gradient calculation
                EG = gamma*EG + (1-gamma)*X[i]*X[i]
                v = lr/np.sqrt(EG + eps)*X[i]
                dw += (a - y[i]) * v.reshape(n, 1)
                db += (a - y[i])
                
            dw /= len(X)
            db /= len(X)
            
            # gradient step
            self.w = self.w - learning_rate * dw
            self.b = self.b - learning_rate * db
            
            # save loss for plotting
            if epoch % self.report_every == 0:
                self.losses_train.append(log_loss(y, self.predict(X)))
                self.losses_test.append(log_loss(y_test, self.predict(X_test)))
    
    def predict(self, X):        
        return np.array([sigmoid(x.reshape(1, n).dot(self.w) + self.b)[0][0] 
                         for x in X])

In [176]:
model = LogisticRegression()
# model.train_grad(X_train, y_train, epochs=100)

In [99]:
test_prediction = np.array(model.predict(X_test))
test_accuracy = np.sum((test_prediction > 0.5) == y_test) / len(test_prediction)
print(f'Точность на тестовой выборке: {round(test_accuracy * 100, 2)}%')

Точность на тестовой выборке: 60.0%


In [100]:
train_prediction = np.array(model.predict(X_train))
train_accuracy = np.sum((train_prediction > 0.5) == y_train) / len(train_prediction)
print(f'Точность на тестовой выборке: {round(train_accuracy * 100, 2)}%')

Точность на тестовой выборке: 47.5%


In [177]:
model.train_RMSProp(X_train, y_train, epochs=100)

In [178]:
test_prediction = np.array(model.predict(X_test))
test_accuracy = np.sum((test_prediction > 0.5) == y_test) / len(test_prediction)
print(f'Точность на тестовой выборке: {round(test_accuracy * 100, 2)}%')

Точность на тестовой выборке: 60.0%


In [179]:
train_prediction = np.array(model.predict(X_train))
train_accuracy = np.sum((train_prediction > 0.5) == y_train) / len(train_prediction)
print(f'Точность на тестовой выборке: {round(train_accuracy * 100, 2)}%')

Точность на тестовой выборке: 47.5%
